# Data Arranging para EDA: Cryptomonedas - Piero Termini

In [57]:
import os
import sys
import requests as rq
import pandas as pd


## Acceso a los datos de cada Cryptomoneda a estudiar mediante Api
### Usando la Api de Coingecko que permite ver la data historica de todas las cripto desde de 2017 sin tener que pagar.

In [58]:
url_btc = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_eth = "https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_bnc = "https://api.coingecko.com/api/v3/coins/binancecoin/market_chart/range?vs_currency=usd&from=1493593200&to=11635156000"
url_ada = "https://api.coingecko.com/api/v3/coins/cardano/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_sol = "https://api.coingecko.com/api/v3/coins/solana/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_xrp = "https://api.coingecko.com/api/v3/coins/ripple/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_usdt = "https://api.coingecko.com/api/v3/coins/tether/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_doge = "https://api.coingecko.com/api/v3/coins/dogecoin/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_dot = "https://api.coingecko.com/api/v3/coins/polkadot/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_shib = "https://api.coingecko.com/api/v3/coins/shiba-inu/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"
url_ltc = "https://api.coingecko.com/api/v3/coins/litecoin/market_chart/range?vs_currency=usd&from=1493593200&to=1635156000"

## Convertir los Datos
### Obtenemos los datos en formato .JSON y los convertiremos en DataFrames de Pandas, luego los agruparemos para poder hacer algunos cambios.

In [59]:
data_btc = rq.get(url_btc).json()
data_eth = rq.get(url_eth).json()
data_bnc = rq.get(url_bnc).json()
data_ada = rq.get(url_ada).json()
data_sol = rq.get(url_sol).json()
data_xrp = rq.get(url_xrp).json()
data_usdt = rq.get(url_usdt).json()
data_doge = rq.get(url_doge).json()
data_dot = rq.get(url_dot).json()
data_shib = rq.get(url_shib).json()
data_ltc = rq.get(url_ltc).json()

In [60]:
hist_btc = pd.DataFrame(data_btc)
hist_eth = pd.DataFrame(data_eth)
hist_bnc = pd.DataFrame(data_bnc)
hist_ada = pd.DataFrame(data_ada)
hist_sol = pd.DataFrame(data_sol)
hist_xrp = pd.DataFrame(data_xrp)
hist_usdt = pd.DataFrame(data_usdt)
hist_doge = pd.DataFrame(data_doge)
hist_dot = pd.DataFrame(data_dot)
hist_ltc = pd.DataFrame(data_ltc)
hist_shib = pd.DataFrame(data_shib)


In [61]:
data_frames = [hist_btc,hist_eth,hist_bnc,hist_ada,hist_usdt,hist_sol,hist_xrp,hist_dot,hist_doge,hist_shib,hist_ltc]

Pasamos una función para aplicar los cambios que queremos a todos los datos, luego corregiremos factores estéticos para mejorar la
visibilidad de la información y ordeanremos cada Dataframe para unirlos facilmente.

In [62]:
hist_btc.head(2) #como podemos ver, cada columna contiene las fechas en formato UNIX y estan repetidas 3 veces, 
                #de los DataFrames a unir, por eso pasaremos una funcion que los ajustará

,prices,market_caps,total_volumes
0,"[1493596800000, 1408.445]","[1493596800000, 22964730936.125]","[1493596800000, 217759983.7243955]"
1,"[1493683200000, 1442.245]","[1493683200000, 23518941607.875]","[1493683200000, 189160669.69719717]"


In [63]:
def fix_df_coingecko(df):
    import pandas as pd
    df["Dates"] = df["prices"]
    df["prices"] = df["prices"].apply(lambda x: x[1])
    df["market_caps"] = df["market_caps"].apply(lambda x: x[1])
    df["total_volumes"] = df["total_volumes"].apply(lambda x: x[1])
    df["Dates"] = df["Dates"].apply(lambda x: x[0])
    df["Dates"] = pd.to_datetime(df["Dates"], unit='ms')

In [64]:
for i in data_frames: #La función fix_df_coingecko toma DataFrames específicos obtenidos de Coingecko, añade Dates como copia 
#de su columna prices, el primer elem e cada columna son fechas en formato UNIX, se elimina el dato extra mediante lamba en
#cada columna y por último, se transforma Dates de Unix a DateTime AAAA-MM-DD.
    fix_df_coingecko(i)

In [65]:
hist_btc.head(2) #nos queda de esta manera.

,prices,market_caps,total_volumes,Dates
0,1408.445,2.296473e+10,2.177600e+08,2017-05-01
1,1442.245,2.351894e+10,1.891607e+08,2017-05-02


In [66]:
#Vamos a renombrar las columnas para dar una mejor estética a los datos y asignaremos Dates como índice, ya que esto los usaremos
#más adelante para unir los DataFrames. Además, debido a que hay datos faltantes, voltearemos el orden de los mismos para evitar
#problemas con los NaN en el Dataframe combinado, para que todas las fechas inicien igual aunque tengan valores nulos.
hist_btc = hist_btc.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_eth = hist_eth.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_bnc = hist_bnc.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_ada = hist_ada.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_sol = hist_sol.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_xrp = hist_xrp.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_usdt = hist_usdt.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_doge = hist_doge.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_dot = hist_dot.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_ltc = hist_ltc.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])
hist_shib = hist_shib.rename(columns={'prices': 'Prices', 'market_caps': 'Market_Cap','total_volumes': 'Total_Volume'}).iloc[::-1].set_index(["Dates"])

In [67]:
hist_btc.head(2) #nos queda de esta manera.

,Prices,Market_Cap,Total_Volume
Dates,,,
2021-10-25,61173.166391,1.153315e+12,2.998254e+10
2021-10-24,61572.140434,1.161166e+12,2.850504e+10


In [68]:
#Declaramos una variable en donde concatenaremos cada Dataframe dentro de un Diccionario
crypto2017_2021 = pd.concat({"Bitcoin":hist_btc, "Ethereum":hist_eth, "Binance_Coin":hist_bnc,"Cardano":hist_ada,"Tether":hist_usdt,"Solana":hist_sol,"Polkadot":hist_dot,
"Ripple":hist_xrp,"Dogecoin":hist_doge,"Shiba_Inu":hist_shib,"Litecoin":hist_ltc}, axis=1)

In [69]:
#Esto nos arrojará un Dataframe con un Multi-Índice, indexado por las fechas y con los 3 valores de cada Crypto
crypto2017_2021.head(3)

Bitcoin                               Ethereum                \
               Prices    Market_Cap  Total_Volume     Prices    Market_Cap   
Dates                                                                        
2017-05-01  1408.4450  2.296473e+10  2.177600e+08  76.846178  7.010408e+09   
2017-05-02  1442.2450  2.351894e+10  1.891607e+08  77.346044  7.058414e+09   
2017-05-03  1485.8175  2.423194e+10  2.061546e+08  80.929492  7.387794e+09   

                         Binance_Coin                         Cardano  ...  \
            Total_Volume       Prices Market_Cap Total_Volume  Prices  ...   
Dates                                                                  ...   
2017-05-01  2.901263e+08          NaN        NaN          NaN     NaN  ...   
2017-05-02  1.387949e+08          NaN        NaN          NaN     NaN  ...   
2017-05-03  1.151840e+08          NaN        NaN          NaN     NaN  ...   

                 Ripple  Dogecoin                             Shiba_Inu  \
           Total_Volume    Prices    Market_Cap  Total_Volume    Prices   
Dates                                                                     
2017-05-01   60236000.0  0.000684  7.463424e+07  3.357178e+06       NaN   
2017-05-02   34435200.0  0.000612  6.685646e+07  3.324582e+06       NaN   
2017-05-03   40572500.0  0.000680  7.426334e+07  2.479327e+06       NaN   

                                     Litecoin                              
           Market_Cap Total_Volume     Prices    Market_Cap  Total_Volume  
Dates                                                                      
2017-05-01        NaN          NaN  15.895431  8.089639e+08  1.023764e+08  
2017-05-02        NaN          NaN  16.023527  8.157943e+08  5.271601e+07  
2017-05-03        NaN          NaN  21.172688  1.078361e+09  3.275667e+08  

[3 rows x 33 columns]

In [70]:
crypto2017_2021.describe()

Bitcoin                                 Ethereum                \
             Prices    Market_Cap  Total_Volume       Prices    Market_Cap   
count   1639.000000  1.639000e+03  1.639000e+03  1639.000000  1.639000e+03   
mean   14900.226919  2.723104e+11  2.137075e+10   708.162532  7.885775e+10   
std    15440.055996  2.918408e+11  2.068267e+10   911.781220  1.072609e+11   
min     1408.445000  2.296473e+10  1.178764e+08    76.846178  7.010408e+09   
25%     6341.862791  1.094502e+11  3.219654e+09   188.943595  2.008741e+10   
50%     8781.175282  1.561772e+11  1.934947e+10   297.069605  2.906110e+10   
75%    12939.558178  2.297752e+11  3.288580e+10   683.741804  6.865463e+10   
max    66237.517550  1.250376e+12  1.788941e+11  4182.790286  4.931254e+11   

                    Binance_Coin                                  Cardano  \
       Total_Volume       Prices    Market_Cap  Total_Volume       Prices   
count  1.639000e+03  1501.000000  1.501000e+03  1.501000e+03  1469.000000   
mean   1.129437e+10    79.853753  1.221392e+10  8.054454e+08     0.391595   
std    1.438208e+10   145.719727  2.278701e+10  1.760848e+09     0.628749   
min    5.911434e+06     0.039818  3.981770e+06  7.178009e-01     0.021321   
25%    1.502943e+09    11.468079  1.401182e+09  6.049705e+07     0.048926   
50%    7.106350e+09    17.400270  2.576607e+09  2.196204e+08     0.095019   
75%    1.464246e+10    30.875864  4.534968e+09  4.584201e+08     0.313286   
max    1.409065e+11   675.098974  1.039651e+11  2.210352e+10     2.967075   

       ...        Ripple     Dogecoin                              \
       ...  Total_Volume       Prices    Market_Cap  Total_Volume   
count  ...  1.639000e+03  1639.000000  1.639000e+03  1.639000e+03   
mean   ...  2.333671e+09     0.039172  5.073096e+09  8.949964e+08   
std    ...  3.217635e+09     0.096966  1.264214e+10  3.554361e+09   
min    ...  2.062220e+07     0.000612  6.685646e+07  1.397688e+05   
25%    ...  4.656590e+08     0.002322  2.778693e+08  1.962615e+07   
50%    ...  1.403940e+09     0.002840  3.430203e+08  6.293765e+07   
75%    ...  2.538031e+09     0.005378  6.163074e+08  1.465303e+08   
max    ...  2.932515e+10     0.681842  8.879477e+10  5.096065e+10   

          Shiba_Inu                                 Litecoin                \
             Prices    Market_Cap  Total_Volume       Prices    Market_Cap   
count  4.370000e+02  4.370000e+02  4.370000e+02  1639.000000  1.639000e+03   
mean   4.285893e-06  1.910289e+09  5.710645e+08    95.882644  5.926244e+09   
std    6.906629e-06  3.240009e+09  1.832596e+09    65.927984  4.201484e+09   
min    5.700414e-11  0.000000e+00  3.702257e-04    15.895431  8.089639e+08   
25%    1.244159e-10  0.000000e+00  6.719149e+02    47.665005  2.942400e+09   
50%    4.873770e-08  0.000000e+00  1.435551e+06    66.208480  4.053232e+09   
75%    7.247955e-06  3.565986e+09  3.722404e+08   134.921142  8.442814e+09   
max    3.599472e-05  1.778382e+10  1.797929e+10   384.671618  2.567776e+10   

                     
       Total_Volume  
count  1.639000e+03  
mean   2.433412e+09  
std    2.518420e+09  
min    3.875832e+07  
25%    4.529543e+08  
50%    1.978824e+09  
75%    3.353926e+09  
max    1.697272e+10  

[8 rows x 33 columns]

In [71]:
crypto2017_2021.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1657 entries, 2017-05-01 00:00:00 to 2021-10-27 00:00:00
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   (Bitcoin, Prices)             1639 non-null   float64
 1   (Bitcoin, Market_Cap)         1639 non-null   float64
 2   (Bitcoin, Total_Volume)       1639 non-null   float64
 3   (Ethereum, Prices)            1639 non-null   float64
 4   (Ethereum, Market_Cap)        1639 non-null   float64
 5   (Ethereum, Total_Volume)      1639 non-null   float64
 6   (Binance_Coin, Prices)        1501 non-null   float64
 7   (Binance_Coin, Market_Cap)    1501 non-null   float64
 8   (Binance_Coin, Total_Volume)  1501 non-null   float64
 9   (Cardano, Prices)             1469 non-null   float64
 10  (Cardano, Market_Cap)         1469 non-null   float64
 11  (Cardano, Total_Volume)       1469 non-null   float64
 12  (Tether, Prices)          

In [72]:
#Guardaremos el DataFrame combinado, se utilizará formato .hdf (aunque tenga menor compatibilidad)
# y no .csv o .xlsl, para conservar la estructura de los datos y el multi-índice.
crypto2017_2021.to_hdf("hdf\Crypto2017_2021.hdf", key="Crypto2017_2021", mode="w") 

## Acceso a los datos generales actuales del mercado nuevamente mediante Api
### Igualmente, la Api de Coingecko permite ver la data general actual de mas de 2000 cryptomonedas

In [73]:
#Por el tamaño de los datos y el enfoque del EDA, la cantidad de información se limitará al Top 200 de cryptomonedas a nivel mundial.
url_top200 = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=200&page=1&sparkline=false"
top_200_crypto = pd.read_json(url_top200)

## Convertir los Datos
### Repetimos los pasos iniciales para los próximos DataFrames que obtendremos de la API

In [74]:
top_200_crypto.head(2)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,59007.00,1110178204206,1,1.236407e+12,42370998257,62790.00,...,21000000.0,21000000.0,67277.00,-12.48611,2021-10-20T14:54:17.702Z,67.810000,86727.02789,2013-07-06T00:00:00.000Z,None,2021-10-27T18:04:36.833Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,4001.82,471412913199,2,NaN,29698879613,4304.67,...,NaN,NaN,4361.18,-8.51456,2021-10-21T11:34:21.968Z,0.432979,921386.98443,2015-10-20T00:00:00.000Z,"{'times': 89.80331567668361, 'currency': 'btc'...",2021-10-27T18:05:31.120Z


In [75]:
#nos quedmaos con los datos relevantes
top_200_crypto = top_200_crypto[["name", 'current_price', 'market_cap', "ath"]]

In [76]:
#Añadimos el Total Supply por cálculo matemático, ya que el Total Supply en la tabla está incompleto, este valor se calcula 
#dividiendo la capitalización de mercado por el precio de cada token.
total_supply = top_200_crypto["market_cap"] / top_200_crypto["current_price"]
top_200_crypto["total_supply"] = total_supply

In [77]:
#Se acomoda la est[etica de la tabla
top_200_crypto = top_200_crypto.rename(columns={'name': 'Names', 'current_price':'Current Price','market_cap': 'Market Cap','ath':'ATH','total_supply': 'Total Supply'})

In [78]:
top_200_crypto.head()

,Names,Current Price,Market Cap,ATH,Total Supply
0,Bitcoin,59007.000000,1110178204206,67277.00,1.881435e+07
1,Ethereum,4001.820000,471412913199,4361.18,1.177996e+08
2,Binance Coin,453.820000,76206658612,686.31,1.679227e+08
3,Tether,0.999755,70488071879,1.32,7.050535e+10
4,Cardano,1.980000,63136183361,3.09,3.188696e+10


In [79]:
#Revisamos la información:
top_200_crypto.info() #No eliminamos los valores Null para no afectar la estrutuctura del DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Names          200 non-null    object 
 1   Current Price  200 non-null    float64
 2   Market Cap     200 non-null    int64  
 3   ATH            200 non-null    float64
 4   Total Supply   200 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 7.9+ KB


In [80]:
#Lo pasamos a formato .hdf
top_200_crypto.to_hdf("hdf\Top200crypto.hdf", key="Top200crypto", mode="w") 

## Acceso a los datos generales de las categorias de cryptomonedas mas comunes
### Nuevamente haremos uso de la Api de Coingecko, por la diversidad de categorías limitaremos su cantidad al Top 10 

In [81]:
url_categorias = "https://api.coingecko.com/api/v3/coins/categories"
categorias_crypto = pd.read_json(url_categorias)
categorias_crypto = pd.DataFrame(categorias_crypto)

In [82]:
categorias_crypto.head(2)

,id,name,market_cap,market_cap_change_24h,content,top_3_coins,volume_24h,updated_at
0,smart-contract-platform,Smart Contract Platform,7.320746e+11,-6.082991,Smart contract platforms are usually blockchai...,[https://assets.coingecko.com/coins/images/279...,5.956055e+10,2021-10-27 18:05:27.814000+00:00
1,decentralized-finance-defi,Decentralized Finance (DeFi),1.450553e+11,-7.152534,None,[https://assets.coingecko.com/coins/images/828...,1.840035e+10,2021-10-27 18:05:26.871000+00:00


In [83]:
#Filtramos y nos quedamos con lo que nos interesa y a su vez lo estilizamos
categorias_crypto = categorias_crypto[["name","market_cap"]].rename(columns={'name': 'Names', 'market_cap': 'Market_Cap'}).set_index(categorias_crypto.index+1).head(10)

In [84]:
categorias_crypto.head(2)

,Names,Market_Cap
1,Smart Contract Platform,7.320746e+11
2,Decentralized Finance (DeFi),1.450553e+11


In [85]:
#Revisamos la información:
categorias_crypto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 1 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Names       10 non-null     object 
 1   Market_Cap  10 non-null     float64
dtypes: float64(1), object(1)
memory usage: 292.0+ bytes


In [86]:
#Nuevamente, lo extraemos en formato.hdf
categorias_crypto.to_hdf("hdf\Categoriascrypto.hdf", key="Top15crypto", mode="w") 

## Acceso a data pública de las empresas privadas con las carteras mas grandes de Bitcoin a nivel mundial
### Usando de la Api de Coingecko

In [87]:
#Nos interesan los valores individuales dentro de la columna companies
url_companias = 'https://api.coingecko.com/api/v3/companies/public_treasury/bitcoin'
bitcoin_holders = pd.read_json(url_companias)
bitcoin_holders = pd.DataFrame(bitcoin_holders)
bitcoin_holders.head(2)


,total_holdings,total_value_usd,market_cap_dominance,companies
0,210237.6658,1.242858e+10,1.12,"{'name': 'MicroStrategy Inc.', 'symbol': 'NASD..."
1,210237.6658,1.242858e+10,1.12,"{'name': 'Tesla', 'symbol': 'NASDAQ: TSLA', 'c..."


In [88]:
bitcoin_holders3 = [] #declaramos una lista vacía donde haremos un append a través de un bucle de las principales
                      #compañías en el DataFrame
for elem in bitcoin_holders["companies"]:
    bitcoin_holders3.append(elem)
bitcoin_holders3 = pd.DataFrame(bitcoin_holders3)
bitcoin_holders = bitcoin_holders3.set_index(bitcoin_holders3.index+1)

In [89]:
bitcoin_holders.head()

,name,symbol,country,total_holdings,total_entry_value_usd,total_current_value_usd,percentage_of_total_supply
1,MicroStrategy Inc.,NASDAQ:MSTR,US,114042,3160000000,6741796706,0.543
2,Tesla,NASDAQ: TSLA,US,48000,1500000000,2837605810,0.229
3,Galaxy Digital Holdings,TSE:GLXY,CA,16402,134000000,969633552,0.078
4,Square Inc.,NASDAQ:SQ,US,8027,220000000,474530455,0.038
5,Marathon Patent Group,NASDAQ:MARA,US,4813,150000000,284529099,0.023


In [90]:
#Nuevamente, nos quedamos con los datos de interés
bitcoin_holders = bitcoin_holders[["name","country","total_holdings","total_current_value_usd","percentage_of_total_supply"]]
bitcoin_holders.head()

,name,country,total_holdings,total_current_value_usd,percentage_of_total_supply
1,MicroStrategy Inc.,US,114042,6741796706,0.543
2,Tesla,US,48000,2837605810,0.229
3,Galaxy Digital Holdings,CA,16402,969633552,0.078
4,Square Inc.,US,8027,474530455,0.038
5,Marathon Patent Group,US,4813,284529099,0.023


In [91]:
#Revisamos la Data
bitcoin_holders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 1 to 27
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        27 non-null     object 
 1   country                     27 non-null     object 
 2   total_holdings              27 non-null     int64  
 3   total_current_value_usd     27 non-null     int64  
 4   percentage_of_total_supply  27 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ KB


In [92]:
bitcoin_holders.groupby("country")["name"].count()

country
AU        1
CA        9
Canada    1
DE        1
HK        1
Japan     1
NO        1
TH        1
UK        2
US        9
Name: name, dtype: int64

In [93]:
bitcoin_holders["country"].replace({"Japan": "JP", "Canada": "CA"}, inplace=True) #arreglamos los valores diferentes

In [94]:
bitcoin_holders.groupby("country")["name"].count()

country
AU     1
CA    10
DE     1
HK     1
JP     1
NO     1
TH     1
UK     2
US     9
Name: name, dtype: int64

In [95]:
#Por último lo pasamos a formato .hdf
bitcoin_holders.to_hdf("hdf\Bitcoin_holders.hdf", key="Bitcoin_holders", mode="w") 